In [50]:
# Author: Peter Prettenhofer <peter.prettenhofer@gmail.com>
#         Lars Buitinck <L.J.Buitinck@uva.nl>
# License: BSD 3 clause

from __future__ import print_function

from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

import pandas as pd


# # Display progress logs on stdout
# logging.basicConfig(level=logging.INFO,
#                     format='%(asctime)s %(levelname)s %(message)s')

# # parse commandline arguments
# op = OptionParser()
# op.add_option("--lsa",
#               dest="n_components", type="int",
#               help="Preprocess documents with latent semantic analysis.")
# op.add_option("--no-minibatch",
#               action="store_false", dest="minibatch", default=True,
#               help="Use ordinary k-means algorithm (in batch mode).")
# op.add_option("--no-idf",
#               action="store_false", dest="use_idf", default=True,
#               help="Disable Inverse Document Frequency feature weighting.")
# op.add_option("--use-hashing",
#               action="store_true", default=False,
#               help="Use a hashing feature vectorizer")
# op.add_option("--n-features", type=int, default=10000,
#               help="Maximum number of features (dimensions)"
#                    " to extract from text.")
# op.add_option("--verbose",
#               action="store_true", dest="verbose", default=False,
#               help="Print progress reports inside k-means algorithm.")

# print(__doc__)
# op.print_help()

# (opts, args) = op.parse_args()
# if len(args) > 0:
#     op.error("this script takes no arguments.")
#     sys.exit(1)

In [51]:
# queries_file = open('/Users/george/Downloads/Queries_cleaned.txt','w')

In [52]:
# with open("/Users/george/Downloads/Queries.txt") as f:
#     for text in f:
#         text = text.replace(';','')
#         text = '"%s"' % text.strip()
#         queries_file.write(text+',')  
        
# queries_file.close()

In [53]:
# id_content_file = open('/Users/george/projects/clustering/id-content','w')


In [54]:
# with open("/Users/george/Downloads/id-content") as f:
#     for text in f:
#         if(text.count(',')==1):
#             text = text.replace(',',';')
#             id_content_file.write(text+'\n')
#         else:
#             backwards = text[::-1]
#             backwards = backwards.replace(',',';',1)
#             text =  backwards[::-1]
#             id_content_file.write(text+'\n')
#             print(text)
        

# id_content_file.close()

In [55]:



dataset = pd.read_csv("/Users/george/Downloads/id-content", header = None, parse_dates="True", sep=";")



In [56]:
dataset

,0,1
0,http://www.theherkimer.com/beer-and-a-bracket/,minneapolis
1,http://www.theherkimer.com/from-the-brewery-14/,minneapolis
2,http://www.theherkimer.com/st-patricks-day-2016/,minneapolis
3,http://www.theherkimer.com/twins-home-opener/,minneapolis
4,http://www.thehousebarbados.com/before-you-arrive,canada
5,http://www.theindychannel.com/about-us/contact-us,mexico
6,http://www.themetalmenders.com/locations.htm,pittsburgh verona
7,http://www.thenation.com/article/rise-new-stud...,washington kronstadt wisconsin brooklyn latin ...
8,http://www.theoxfordhotel.com/,denver
9,http://www.thestreet.com/author/1101023/TedRee...,miami alaska


In [57]:
dataset
document_list = []
dataset.columns = ['id', 'doc']
dataset['doc'].fillna(value='travel', inplace=True)



document_list = dataset['doc'].tolist()
id_list = dataset['id'].tolist()

In [58]:
len(dataset)

85354

In [59]:
labels = dataset['id'].tolist()
true_k = np.unique(labels).shape[0]

print("Extracting features from the training dataset using a sparse vectorizer")
t0 = time()
# if opts.use_hashing:
#     if opts.use_idf:
#         # Perform an IDF normalization on the output of HashingVectorizer
#         hasher = HashingVectorizer(n_features=opts.n_features,
#                                    stop_words='english', non_negative=True,
#                                    norm=None, binary=False)
#         vectorizer = make_pipeline(hasher, TfidfTransformer())
#     else:
#         vectorizer = HashingVectorizer(n_features=opts.n_features,
#                                        stop_words='english',
#                                        non_negative=False, norm='l2',
#                                        binary=False)
# else:
#     vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
#                                  min_df=2, stop_words='english',
#                                  use_idf=opts.use_idf)

vectorizer = TfidfVectorizer(max_df=0.5,
                                 min_df=2, stop_words='english',
                                 use_idf=True)
X = vectorizer.fit_transform(document_list)

print("done in %fs" % (time() - t0))
print("n_samples: %d, n_features: %d" % X.shape)
print()


print("Performing dimensionality reduction using LSA")
t0 = time()
# Vectorizer results are normalized, which makes KMeans behave as
# spherical k-means for better results. Since LSA/SVD results are
# not normalized, we have to redo the normalization.
svd = TruncatedSVD()
normalizer = Normalizer(copy=False)
lsa = make_pipeline(svd, normalizer)

X = lsa.fit_transform(X)

print("done in %fs" % (time() - t0))

explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(
    int(explained_variance * 100)))

print()



Extracting features from the training dataset using a sparse vectorizer
done in 2.021080s
n_samples: 85354, n_features: 3950

Performing dimensionality reduction using LSA
done in 0.359440s
Explained variance of the SVD step: 16%



In [60]:

###############################################################################
# Do the actual clustering

# if opts.minibatch:
#     km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
#                          init_size=1000, batch_size=1000)
# else:
km = KMeans(n_clusters=300, init='k-means++', max_iter=10000, n_init=1)

print("Clustering sparse data with %s" % km)
t0 = time()
km.fit(X)
print("done in %0.3fs" % (time() - t0))
print()

# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km.labels_))
# print("Completeness: %0.3f" % metrics.completeness_score(labels, km.labels_))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels, km.labels_))
# print("Adjusted Rand-Index: %.3f"
#       % metrics.adjusted_rand_score(labels, km.labels_))
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, km.labels_, sample_size=1000))

# print()


# if not opts.use_hashing:
#     print("Top terms per cluster:")

#     if opts.n_components:
#         original_space_centroids = svd.inverse_transform(km.cluster_centers_)
#         order_centroids = original_space_centroids.argsort()[:, ::-1]
#     else:
#         order_centroids = km.cluster_centers_.argsort()[:, ::-1]

#     terms = vectorizer.get_feature_names()
#     for i in range(true_k):
#         print("Cluster %d:" % i, end='')
#         for ind in order_centroids[i, :10]:
#             print(' %s' % terms[ind], end='')
#         print()

Clustering sparse data with KMeans(copy_x=True, init='k-means++', max_iter=10000, n_clusters=300,
    n_init=1, n_jobs=1, precompute_distances='auto', random_state=None,
    tol=0.0001, verbose=0)
done in 2.115s



In [61]:
km.labels_

array([211, 211, 211, ...,  81, 121, 289], dtype=int32)

In [62]:
document_Series = pd.Series(document_list)
cluster_Series = pd.Series(km.labels_)
id_Series = pd.Series(id_list)

In [63]:
columns_to_export=(pd.concat([id_Series,cluster_Series], axis=1)).columns
results = (pd.concat([id_Series,cluster_Series], axis=1))
results.columns = ['id', 'doc']

In [64]:
results['doc']=results['doc']+1
results.to_csv("Clustered_results.txt", sep='\t', columns=['id', 'doc'] ,header=False,index=False,encoding='utf-8')

In [65]:
cluster_Series.value_counts()

1      4939
205    3767
4      3313
13     2427
0      2187
55     1970
292    1874
252    1856
138    1846
142    1679
39     1574
28     1395
120    1306
206    1280
24     1269
128    1072
262    1070
268    1056
164    1055
175    1001
80     1001
160     977
78      967
47      889
79      859
38      849
33      782
214     752
74      749
223     730
       ... 
110       9
260       9
190       8
264       8
46        8
11        8
141       8
191       7
99        7
133       6
83        6
177       6
249       6
98        6
258       6
287       6
161       6
202       5
266       5
274       5
256       4
18        4
228       3
32        3
170       2
152       2
203       2
192       1
67        1
123       1
dtype: int64

In [66]:
results

,id,doc
0,http://www.theherkimer.com/beer-and-a-bracket/,212
1,http://www.theherkimer.com/from-the-brewery-14/,212
2,http://www.theherkimer.com/st-patricks-day-2016/,212
3,http://www.theherkimer.com/twins-home-opener/,212
4,http://www.thehousebarbados.com/before-you-arrive,143
5,http://www.theindychannel.com/about-us/contact-us,40
6,http://www.themetalmenders.com/locations.htm,59
7,http://www.thenation.com/article/rise-new-stud...,132
8,http://www.theoxfordhotel.com/,157
9,http://www.thestreet.com/author/1101023/TedRee...,9
